In [9]:
import boto3
import json
import pandas as pd

In [14]:
# setup stuff

def load_config():
    with open('../config.json', 'r') as file:
        config = json.load(file)
    return config

config = load_config()

def display_config(aws_access_key_id, aws_secret_access_key):
    """Displays masked AWS credentials."""
    def mask_string(s):
        """Helper function to mask the middle part of a string."""
        if len(s) > 6:
            return s[:3] + '*' * (len(s) - 6) + s[-3:]
        else:
            return '*' * len(s)

    print("AWS Access Key ID: ", mask_string(aws_access_key_id))
    print("AWS Secret Access Key: ", mask_string(aws_secret_access_key))


display_config(config['AWS_ACCESS_KEY_ID'], config['AWS_SECRET_ACCESS_KEY'])

AWS Access Key ID:  AKI**************NHT
AWS Secret Access Key:  RBI**********************************r45


In [18]:
config.keys()

dict_keys(['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'BUCKET_NAME', 'BASE_PREFIX', 'DATA_DIRECTORY', 'P_YM'])

In [ ]:
# /src/aws_utils.py

